In [1]:
import copy
import grakel
from grakel.kernels import WeisfeilerLehman, VertexHistogram, ShortestPath, SubgraphMatching
import networkx as nx
import numpy as np
import pandas as pd
from rdflib import Graph, URIRef
from rdflib.extras.external_graph_libs import rdflib_to_networkx_digraph, rdflib_to_networkx_multidigraph
from rdflib.namespace import DefinedNamespace, Namespace, RDF, RDFS, SOSA
from rdflib.term import Literal, URIRef
from sklearn.model_selection import KFold
from tqdm import tqdm

from viscars.data import DataLoader
from viscars.evaluation.evaluators import Evaluator
from viscars.evaluation.metrics import MetricType
from viscars.evaluation.metrics.factory import MetricFactory
from viscars.recommenders import Recommender
from viscars.recommenders.factory import RecommenderFactory, RecommenderType
from viscars.utils import visualize_graph

In [2]:
class DASHB(DefinedNamespace):
    """
    The Dynamic Dashboard Ontology
    """

    _fail = True

    # Classes
    User: URIRef
    UserGroup: URIRef
    Dashboard: URIRef
    Tab: URIRef
    Widget: URIRef
    Property: URIRef
    Metric: URIRef
    Visualization: URIRef
    ObservationBoundary: URIRef

    # User
    memberOf: URIRef

    # Dashboard
    hasTab: URIRef

    # Tab
    hasWidget: URIRef

    # Widget
    createdBy: URIRef
    hasProperty: URIRef
    visualizedBy: URIRef
    hasObservationBoundary: URIRef

    # Property
    produces: URIRef

    # ObservationBoundary
    hasMinBoundary: Literal
    hasMaxBoundary: Literal
    hasBoundaryLabel: Literal

    _NS = Namespace('http://dynamicdashboard.ilabt.imec.be/broker/ontologies/dashboard#')

In [3]:
graph = Graph()
graph.parse(f'./data/protego/graph.ttl', format='n3')
graph.parse(f'./data/protego/protego_zplus.ttl', format='n3')

<Graph identifier=N9715ccdc35f343f4a7fa387760f78a91 (<class 'rdflib.graph.Graph'>)>

## Generate (sub)graphs

In [4]:
def generate_subgraph(graph: Graph, uid: URIRef) -> Graph:
    sub_graph = Graph()
    sub_graph += graph.triples((uid, None, None))

    for widget in graph.subjects(DASHB.createdBy, uid):       
        sub_graph += graph.triples((widget, None, None))
        
        for property_ in graph.objects(widget, DASHB.hasProperty):
            sub_graph += graph.triples((property_, None, None))
            
        for visualization in graph.objects(widget, DASHB.visualizedBy):
            sub_graph += graph.triples((visualization, None, None))
            
        for observation_boundary in graph.objects(widget, DASHB.hasObservationBoundary):
            sub_graph += graph.triples((observation_boundary, None, None))
            
    return sub_graph     

In [5]:
users = [str(user) for user in graph.subjects(RDF.type, DASHB.User)]

sub_graphs = {uid: generate_subgraph(graph, URIRef(uid)) for uid in users}

<Graph identifier=N6b345e04329f47699cc0cc1d3270f6f7 (<class 'rdflib.graph.Graph'>)>

## Graph comparison

In [6]:
G_nx = []
for uid_, graph_ in sub_graphs.items():
    networkx_graph = rdflib_to_networkx_digraph(graph_, edge_attrs=lambda s, p, o: {'label': p})    
    # Add node labels
    for id_, data in networkx_graph.nodes(data=True):
        data['label'] = str(id_)
    
    G_nx.append(networkx_graph)

graphs = list(grakel.utils.graph_from_networkx(G_nx, node_labels_tag='label', edge_labels_tag='label', edge_weight_tag='weight'))

In [7]:
uu_similarities = {}
for idx, uid in enumerate(tqdm(users)):
    gk = WeisfeilerLehman(n_iter=2, normalize=True, base_graph_kernel=VertexHistogram)
    check = gk.fit_transform([graphs[idx]])[0]
    
    assert (check == 1)
    
    uu_similarities[uid] = {}
    for idx_, uid_ in enumerate(users):
        if uid == uid_:
            uu_similarities[uid][uid_] = float(check)
        if uid_ in uu_similarities.keys():
            uu_similarities[uid][uid_] = uu_similarities[uid_][uid]
        else:
            uu_similarities[uid][uid_] = float(gk.transform([graphs[idx_]])[0])

100%|████████████████████████████████████████████| 4/4 [00:00<00:00, 153.53it/s]


In [8]:
print(uu_similarities)

{'https://dynamicdashboard.ilabt.imec.be/users/10': {'https://dynamicdashboard.ilabt.imec.be/users/10': 1.0, 'https://dynamicdashboard.ilabt.imec.be/users/8': 0.37080992435478316, 'https://dynamicdashboard.ilabt.imec.be/users/9': 0.36255427673212065, 'https://dynamicdashboard.ilabt.imec.be/users/7': 0.3780443724864984}, 'https://dynamicdashboard.ilabt.imec.be/users/8': {'https://dynamicdashboard.ilabt.imec.be/users/10': 0.37080992435478316, 'https://dynamicdashboard.ilabt.imec.be/users/8': 1.0, 'https://dynamicdashboard.ilabt.imec.be/users/9': 0.3360968098238519, 'https://dynamicdashboard.ilabt.imec.be/users/7': 0.35045651291117486}, 'https://dynamicdashboard.ilabt.imec.be/users/9': {'https://dynamicdashboard.ilabt.imec.be/users/10': 0.36255427673212065, 'https://dynamicdashboard.ilabt.imec.be/users/8': 0.3360968098238519, 'https://dynamicdashboard.ilabt.imec.be/users/9': 1.0, 'https://dynamicdashboard.ilabt.imec.be/users/7': 0.3313577240515163}, 'https://dynamicdashboard.ilabt.imec.be

In [9]:
def kneighbours_cc(uid, cid, iid, k=20, strict=True):
    cc_similarities_ = copy.deepcopy(cc_similarities)
    
    if strict:
        for c, w in cc_similarities[cid].items():
            qry = f'''
                PREFIX dashb: <http://dynamicdashboard.ilabt.imec.be/broker/ontologies/dashboard#>
                PREFIX sosa: <http://www.w3.org/ns/sosa/>
                PREFIX ssn-ext: <http://dynamicdashboard.ilabt.imec.be/broker/ontologies/ssn-extension/>

                SELECT * WHERE {{
                    ?widget a dashb:Widget ;
                        dashb:createdBy <{uid}> ;
                        dashb:hasProperty ?property .
                        
                    ?sensor ssn-ext:subSystemOf <{c}> ;
                        sosa:observes ?property .
                }}
            '''
            if len(list(graph.query(qry))) == 0:
                cc_similarities_[cid].pop(c)
    
    if cid in cc_similarities_[cid].keys():
        cc_similarities_[cid].pop(cid)
    
    return sorted(cc_similarities_[cid].items(), key=lambda x: x[1])[:k]

def kneighbours_ii(iid, k=20):
    ii_similarities_ = copy.deepcopy(ii_similarities)
    ii_similarities_[iid].pop(uid)
    return sorted(ii_similarities_[iid].items(), key=lambda x: x[1])[:k]

def kneighbours_uu(uid, cid, iid, k=20):
    uu_similarities_ = copy.deepcopy(uu_similarities)
                
    if uid in uu_similarities_[uid].keys():
        uu_similarities_[uid].pop(uid)
                
    return sorted(uu_similarities_[uid].items(), key=lambda x: x[1])[:k]

In [10]:
def predict_uucf(uid, cid, iid):
    # Calculate K-nearest neighbours of the user
    neighbours = kneighbours_uu(uid, cid, iid)
    
    total_rw = 0
    total_w = 0
    for n, w in neighbours:
        # Get ratings from user n for item i
        qry = f'''
            PREFIX dashb:<http://dynamicdashboard.ilabt.imec.be/broker/ontologies/dashboard#>
    
            SELECT * WHERE {{
                ?widget a dashb:Widget ;
                    dashb:createdBy <{n}> ;
                    dashb:hasProperty ?property ;
                    dashb:visualizedBy <{iid}> .
                
                <{cid}> dashb:produces ?metric .
                ?property dashb:produces ?metric .
            }}
        '''
        r_aui = 1 if len(list(graph.query(qry))) > 0 else 0
        w_au = w  # Similarity score of user u and user n
        total_rw += r_aui * w_au
        total_w += w
    return total_rw / total_w if total_w > 0 else 0

In [11]:
uid = 'https://dynamicdashboard.ilabt.imec.be/users/10'
uid = 'https://dynamicdashboard.ilabt.imec.be/users/7'

# cid = 'https://webthing.protego.dynamicdashboard.ilabt.imec.be/things/zplus_6.lifestyle/properties/enriched-call'
cid = 'https://webthing.protego.dynamicdashboard.ilabt.imec.be/things/zplus_6.60%253A77%253A71%253A7D%253A93%253AD7%252Fservice0009/properties/org.dyamand.types.health.GlucoseLevel'
cid = 'https://webthing.protego.dynamicdashboard.ilabt.imec.be/things/zplus_146.lifestyle/properties/enriched-call'

items = set(graph.objects(None, DASHB.visualizedBy))
for iid in items:
    p = predict_uucf(uid, cid, iid)
    print(f'[{p}] {iid}')

[0.0] http://localhost/web-thing-gateway/things/visualizations/time-series-line-chart-with-time-range-selector
[0.0] http://localhost/web-thing-gateway/things/visualizations/colored-measurement
[1.0] http://localhost/web-thing-gateway/things/visualizations/enriched-call


In [12]:
# def predict_cccf(uid, cid, iid):
#     # Calculate K-nearest neighbours of the patient
#     neighbours = kneighbours_cc(uid, cid, iid)
    
#     total_rw = 0
#     total_w = 0
#     for n, w in neighbours:
#         # Get ratings from user i for similar items (metric-based) of patient n
#         qry = f'''
#             PREFIX dashb: <http://dynamicdashboard.ilabt.imec.be/broker/ontologies/dashboard#>
#             PREFIX sosa: <http://www.w3.org/ns/sosa/>
#             PREFIX ssn-ext: <http://dynamicdashboard.ilabt.imec.be/broker/ontologies/ssn-extension/>
    
#             SELECT * WHERE {{
#                 <{iid}> dashb:produces ?metric .
                
#                 ?sensor ssn-ext:subSystemOf <{n}> ;
#                     sosa:observes ?property .
#                 ?property dashb:produces ?metric .
                
#                 ?widget a dashb:Widget ;
#                     dashb:createdBy <{uid}> ;
#                     dashb:hasProperty ?property .
#             }}
#         '''
#         r_aui = 1 if len(list(graph.query(qry))) > 0 else 0
#         w_au = w  # Similarity score of patient cid and patient n
#         total_rw += r_aui * w_au
#         total_w += w
#     return total_rw / total_w if total_w > 0 else 0

In [13]:
# for iid in items:
#     p = predict_cccf(uid, cid, iid)
#     print(f'[{p}] {iid}')

In [14]:
def predict(uid, cid):
    items = set(graph.objects(None, DASHB.visualizedBy))
    
    scores = {}
    for iid in items:
        scores[iid] = predict_uucf(uid, cid, iid)
        
    recommendations = [{'contextId': cid, 'itemId': item, 'score': score} for item, score in scores.items()]
    return sorted(recommendations, key=lambda n: n['score'], reverse=True)

In [15]:
predict(uid, cid)

[{'contextId': 'https://webthing.protego.dynamicdashboard.ilabt.imec.be/things/zplus_146.lifestyle/properties/enriched-call',
  'itemId': rdflib.term.URIRef('http://localhost/web-thing-gateway/things/visualizations/enriched-call'),
  'score': 1.0},
 {'contextId': 'https://webthing.protego.dynamicdashboard.ilabt.imec.be/things/zplus_146.lifestyle/properties/enriched-call',
  'itemId': rdflib.term.URIRef('http://localhost/web-thing-gateway/things/visualizations/time-series-line-chart-with-time-range-selector'),
  'score': 0.0},
 {'contextId': 'https://webthing.protego.dynamicdashboard.ilabt.imec.be/things/zplus_146.lifestyle/properties/enriched-call',
  'itemId': rdflib.term.URIRef('http://localhost/web-thing-gateway/things/visualizations/colored-measurement'),
  'score': 0.0}]

# Evaluation

In [16]:
qry = '''
    PREFIX dashb: <http://dynamicdashboard.ilabt.imec.be/broker/ontologies/dashboard#>
    PREFIX sosa: <http://www.w3.org/ns/sosa/>
    PREFIX ssn-ext: <http://dynamicdashboard.ilabt.imec.be/broker/ontologies/ssn-extension/>

    SELECT ?user ?context ?item WHERE {          
        ?widget dashb:hasProperty ?context ;
                dashb:visualizedBy ?item ;
                dashb:createdBy ?user .                
    }
'''

ratings = {'user': [], 'item': [], 'rating': [], 'context': []}
for row in graph.query(qry):
    user_ = row[0]
    context_ = row[1]
    item_ = row[2]
    
    ratings.get('user').append(user_)
    ratings.get('item').append(item_)
    ratings.get('rating').append(5.0)
    ratings.get('context').append(context_)
ratings_df = pd.DataFrame.from_dict(ratings)

In [17]:
ratings_df.head()

,user,item,rating,context
0,https://dynamicdashboard.ilabt.imec.be/users/7,http://localhost/web-thing-gateway/things/visu...,5.0,https://webthing.protego.dynamicdashboard.ilab...
1,https://dynamicdashboard.ilabt.imec.be/users/7,http://localhost/web-thing-gateway/things/visu...,5.0,https://webthing.protego.dynamicdashboard.ilab...
2,https://dynamicdashboard.ilabt.imec.be/users/7,http://localhost/web-thing-gateway/things/visu...,5.0,https://webthing.protego.dynamicdashboard.ilab...
3,https://dynamicdashboard.ilabt.imec.be/users/7,http://localhost/web-thing-gateway/things/visu...,5.0,https://webthing.protego.dynamicdashboard.ilab...
4,https://dynamicdashboard.ilabt.imec.be/users/7,http://localhost/web-thing-gateway/things/visu...,5.0,https://webthing.protego.dynamicdashboard.ilab...


In [18]:
# Context metadata
qry = '''
    PREFIX dashb: <http://dynamicdashboard.ilabt.imec.be/broker/ontologies/dashboard#>
    PREFIX sosa: <http://www.w3.org/ns/sosa/>

    SELECT ?context WHERE {
        ?widget dashb:hasProperty ?context .
    }
'''
context_metadata = {'id': []}

result = graph.query(qry)
for row in result:
    context_metadata.get('id').append(row[0])
context_metadata_df = pd.DataFrame.from_dict(context_metadata)

# User metadata
qry = '''
    PREFIX dashb: <http://dynamicdashboard.ilabt.imec.be/broker/ontologies/dashboard#>

    SELECT ?user ?role WHERE {
        ?user dashb:memberOf ?role .
    }
'''
user_metadata = {'id': [], 'type': []}

result = graph.query(qry)
for row in result:
    user_metadata.get('id').append(row[0])
    user_metadata.get('type').append(row[1])
user_metadata_df = pd.DataFrame.from_dict(user_metadata)

# Item metadata
qry = '''
    PREFIX dashb: <http://dynamicdashboard.ilabt.imec.be/broker/ontologies/dashboard#>

    SELECT ?item WHERE {
        ?widget dashb:visualizedBy ?item .
    }
'''
item_metadata = {'id': []}

result = graph.query(qry)
for row in result:
    item_metadata.get('id').append(row[0])
item_metadata_df = pd.DataFrame.from_dict(item_metadata)


def build_subgraph_from_ratings(ratings: pd.DataFrame) -> Graph:
    sub_graph = Graph()

    for idx, row in context_metadata_df.iterrows():
        cid = row['id']
        sub_graph += graph.triples((cid, None, None))

    for idx, row in user_metadata_df.iterrows():
        uid = row['id']
        sub_graph += graph.triples((uid, None, None))

    for idx, row in item_metadata_df.iterrows():
        iid = row['id']
        sub_graph += graph.triples((iid, None, None))

    for idx, row in ratings.iterrows():
        uid = row['user']
        iid = row['item']
        cid = row['context']

        sub_graph += graph.triples((uid, None, None))
        sub_graph += graph.triples((None, None, uid))
        sub_graph += graph.triples((iid, None, None))
        sub_graph += graph.triples((None, None, iid))

        sub_graph += graph.triples((cid, None, None))
        sub_graph += graph.triples((None, None, cid))

    return sub_graph

In [19]:
class CF(Recommender):
    def __init__(self, graph: Graph, verbose=False):
        super().__init__(graph, verbose)

    def _build_model(self):
        pass
    
    def kneighbours_uu(self, uid, cid, iid, k=20):
        uu_similarities_ = copy.deepcopy(uu_similarities)

        if uid in uu_similarities_[uid].keys():
            uu_similarities_[uid].pop(uid)

        return sorted(uu_similarities_[uid].items(), key=lambda x: x[1])[:k]
    
    def predict_uucf(self, uid, cid, iid):
        # Calculate K-nearest neighbours of the user
        neighbours = kneighbours_uu(uid, cid, iid)

        total_rw = 0
        total_w = 0
        for n, w in neighbours:
            # Get ratings from user n for item i
            qry = f'''
                PREFIX dashb:<http://dynamicdashboard.ilabt.imec.be/broker/ontologies/dashboard#>

                SELECT * WHERE {{
                    ?widget
                        dashb:createdBy <{n}> ;
                        dashb:hasProperty ?property ;
                        dashb:visualizedBy <{iid}> .

                    <{cid}> dashb:produces ?metric .
                    ?property dashb:produces ?metric .
                }}
            '''
            r_aui = 1 if len(list(self.graph.query(qry))) > 0 else 0
            w_au = w  # Similarity score of user u and user n
            total_rw += r_aui * w_au
            total_w += w
        return total_rw / total_w if total_w > 0 else 0

    def predict(self, uid, cid, *kwargs):
        items = set(graph.objects(None, DASHB.visualizedBy))

        scores = {}
        for iid in items:
            scores[iid] = self.predict_uucf(uid, cid, iid)

        recommendations = [{'contextId': cid, 'itemId': item, 'score': score} for item, score in scores.items()]
        return sorted(recommendations, key=lambda n: n['score'], reverse=True)

    def top_n(self, uid: [], cid: [], n: int, **kwargs):
        pass

In [25]:
class KFoldCrossValidation():

    def __init__(self, recommender: Recommender, metrics: [], k=5):
        """
        :param project_id: ID of the project (to load the correct data).
        :param recommender: Recommender
        :param metrics: List of Metrics
        :param k: Number of folds
        """
        self.recommender = recommender
        self.metrics = metrics
        self.k = k

    def evaluate(self, ratings, **kwargs):
        kf = KFold(n_splits=self.k, shuffle=True)

        n_fold = 0

        result = {'folds': [], 'result': {}}
        for train_idx, test_idx in kf.split(ratings):
            train = ratings.iloc[train_idx]
            test = ratings.iloc[test_idx]

            graph = build_subgraph_from_ratings(train)
            self.recommender.set_graph(graph)

            fold_scores = {}

            for uid in tqdm(test['user'].unique()):
                df_user = test.loc[test['user'] == uid]

                for cid in df_user['context']:
                    predictions = self.recommender.predict(str(uid), str(cid), **kwargs)
                    recommendations = [r['itemId'] for r in predictions]

                    truth = []
                    t_user = test.loc[test['user'] == uid]
                    for idx, row in t_user.iterrows():
                        if row['context'] == cid:
                            truth.append(row['item'])
                    # truth = list(test.loc[test['user'] == uid].loc[test['context'] == cid]['item'])
                    
                    print(recommendations)
                    print(truth)

                    for metric in self.metrics:
                        if str(metric) not in fold_scores.keys():
                            fold_scores[str(metric)] = []
                        score = metric.calculate(recommendations, truth)
                        fold_scores[str(metric)].append(score)

            result_for_fold = {}
            for metric, scores in fold_scores.items():
                avg = sum(scores) / len(scores)
                result_for_fold[metric] = avg

                if metric not in result['result'].keys():
                    result['result'][metric] = []
                result['result'][metric].append(avg)
            result['folds'].append(result_for_fold)

            n_fold += 1

        final_results = {}
        for metric_type, score in result['result'].items():
            final_results[metric_type] = sum(score) / len(score)

        result['result'] = final_results
        return result

In [21]:
metric_factory = MetricFactory()

metrics = ['f1@1', 'ndcg@1', 'ndcg@3']
parsed_metrics = []
for metric in metrics:
    m_split = metric.split('@')
    m_type = m_split[0]
    n = int(m_split[1]) if len(m_split) >= 2 else None

    metric_ = metric_factory.get(MetricType.reverse_lookup(m_type), n)
    parsed_metrics.append(metric_)

In [22]:
recommender = CF(graph)

evaluator = KFoldCrossValidation(recommender, metrics=parsed_metrics, k=5)
result = evaluator.evaluate(ratings_df)

for fold in result['folds']:
    print(fold)
print(result['result'])

100%|█████████████████████████████████████████████| 4/4 [00:00<00:00,  5.26it/s]

{'f1@1': 1.0, 'ndcg@1': 1.0, 'ndcg@3': 1.0}
{'f1@1': 0.9, 'ndcg@1': 0.9, 'ndcg@3': 0.9630929753571458}
{'f1@1': 0.8, 'ndcg@1': 0.8, 'ndcg@3': 0.9261859507142916}
{'f1@1': 0.7, 'ndcg@1': 0.7, 'ndcg@3': 0.8892789260714373}
{'f1@1': 0.8888888888888888, 'ndcg@1': 0.8888888888888888, 'ndcg@3': 0.9589921948412731}
{'f1@1': 0.857777777777778, 'ndcg@1': 0.857777777777778, 'ndcg@3': 0.9475100093968296}


In [23]:
class NetworkXPersonalizedPageRank(Recommender):

    def __init__(self, graph: Graph, verbose=False, alpha=0.8, tol=10e-6):
        super().__init__(graph, verbose)

        self.alpha = alpha
        self.tolerance = tol
        self.personalization = None

    def _build_model(self):
        graph_ = Graph()
        graph_ += self.graph.triples((None, None, None))

        self.items = items = set(graph.objects(None, DASHB.visualizedBy))
        self.model = rdflib_to_networkx_multidigraph(graph_).to_undirected()

    def set_personalization(self, weight_uid=0, weight_cid=0):
        weight_others = 1 - weight_uid - weight_cid

        if weight_uid == 0:
            weight_uid = weight_others
        if weight_cid == 0:
            weight_cid = weight_others

        self.personalization = (weight_uid, weight_cid, weight_others)

    def _personalization(self, uid, cid):
        personalization = {}

        for node in self.model.nodes:
            uri = str(node)
            if uri in uid:
                personalization[node] = self.personalization[0]
            elif uri in cid:
                personalization[node] = self.personalization[1]
            else:
                personalization[node] = self.personalization[2]

        return personalization

    def run(self, uid: [] = None, cid: [] = None):
        if uid is not None and cid is not None:  # TODO: Check if UID and CID exist in graph -> ZeroDivisionError
            weights = self._personalization(uid, cid)
            return nx.pagerank(self.model, alpha=self.alpha, personalization=weights,
                               tol=self.tolerance)

        return nx.pagerank(self.model, alpha=self.alpha, tol=self.tolerance)

    def predict(self, uid: [] = None, cid: [] = None, **kwargs):
        pr = self.run(uid, cid)

        recommendations = \
            [{'contextId': cid, 'itemId': item, 'score': p} for item, p in pr.items()
             if item in self.items]
        return sorted(recommendations, key=lambda n: n['score'], reverse=True)

    def top_n(self, uid: [], cid: [], n: int, **kwargs):
        pass


In [26]:
recommender = NetworkXPersonalizedPageRank(graph, alpha=0.3)
recommender.set_personalization(0.7, 0.3)

evaluator = KFoldCrossValidation(recommender, metrics=parsed_metrics, k=5)
result = evaluator.evaluate(ratings_df)

for fold in result['folds']:
    print(fold)
print(result['result'])

100%|████████████████████████████████████████████| 4/4 [00:00<00:00, 102.79it/s]


[rdflib.term.URIRef('http://localhost/web-thing-gateway/things/visualizations/colored-measurement'), rdflib.term.URIRef('http://localhost/web-thing-gateway/things/visualizations/enriched-call'), rdflib.term.URIRef('http://localhost/web-thing-gateway/things/visualizations/time-series-line-chart-with-time-range-selector')]
[rdflib.term.URIRef('http://localhost/web-thing-gateway/things/visualizations/enriched-call')]
[rdflib.term.URIRef('http://localhost/web-thing-gateway/things/visualizations/colored-measurement'), rdflib.term.URIRef('http://localhost/web-thing-gateway/things/visualizations/enriched-call'), rdflib.term.URIRef('http://localhost/web-thing-gateway/things/visualizations/time-series-line-chart-with-time-range-selector')]
[rdflib.term.URIRef('http://localhost/web-thing-gateway/things/visualizations/colored-measurement')]
[rdflib.term.URIRef('http://localhost/web-thing-gateway/things/visualizations/colored-measurement'), rdflib.term.URIRef('http://localhost/web-thing-gateway/th

100%|█████████████████████████████████████████████| 4/4 [00:00<00:00, 79.57it/s]


[rdflib.term.URIRef('http://localhost/web-thing-gateway/things/visualizations/colored-measurement'), rdflib.term.URIRef('http://localhost/web-thing-gateway/things/visualizations/enriched-call'), rdflib.term.URIRef('http://localhost/web-thing-gateway/things/visualizations/time-series-line-chart-with-time-range-selector')]
[rdflib.term.URIRef('http://localhost/web-thing-gateway/things/visualizations/colored-measurement')]
[rdflib.term.URIRef('http://localhost/web-thing-gateway/things/visualizations/colored-measurement'), rdflib.term.URIRef('http://localhost/web-thing-gateway/things/visualizations/enriched-call'), rdflib.term.URIRef('http://localhost/web-thing-gateway/things/visualizations/time-series-line-chart-with-time-range-selector')]
[rdflib.term.URIRef('http://localhost/web-thing-gateway/things/visualizations/colored-measurement')]
[rdflib.term.URIRef('http://localhost/web-thing-gateway/things/visualizations/colored-measurement'), rdflib.term.URIRef('http://localhost/web-thing-gate

  0%|                                                     | 0/4 [00:00<?, ?it/s]

...

[rdflib.term.URIRef('http://localhost/web-thing-gateway/things/visualizations/enriched-call'), rdflib.term.URIRef('http://localhost/web-thing-gateway/things/visualizations/time-series-line-chart-with-time-range-selector'), rdflib.term.URIRef('http://localhost/web-thing-gateway/things/visualizations/colored-measurement')]
[rdflib.term.URIRef('http://localhost/web-thing-gateway/things/visualizations/enriched-call')]
[rdflib.term.URIRef('http://localhost/web-thing-gateway/things/visualizations/colored-measurement'), rdflib.term.URIRef('http://localhost/web-thing-gateway/things/visualizations/enriched-call'), rdflib.term.URIRef('http://localhost/web-thing-gateway/things/visualizations/time-series-line-chart-with-time-range-selector')]
[rdflib.term.URIRef('http://localhost/web-thing-gateway/things/visualizations/enriched-call')]
[rdflib.term.URIRef('http://localhost/web-thing-gateway/things/visualizations/colored-measurement'), rdflib.term.URIRef('http://localhost/web-thing-gateway/things/v

  0%|                                                     | 0/4 [00:00<?, ?it/s]

[rdflib.term.URIRef('http://localhost/web-thing-gateway/things/visualizations/colored-measurement'), rdflib.term.URIRef('http://localhost/web-thing-gateway/things/visualizations/enriched-call'), rdflib.term.URIRef('http://localhost/web-thing-gateway/things/visualizations/time-series-line-chart-with-time-range-selector')]
[rdflib.term.URIRef('http://localhost/web-thing-gateway/things/visualizations/colored-measurement')]
[rdflib.term.URIRef('http://localhost/web-thing-gateway/things/visualizations/colored-measurement'), rdflib.term.URIRef('http://localhost/web-thing-gateway/things/visualizations/enriched-call'), rdflib.term.URIRef('http://localhost/web-thing-gateway/things/visualizations/time-series-line-chart-with-time-range-selector')]
[rdflib.term.URIRef('http://localhost/web-thing-gateway/things/visualizations/enriched-call')]
[rdflib.term.URIRef('http://localhost/web-thing-gateway/things/visualizations/colored-measurement'), rdflib.term.URIRef('http://localhost/web-thing-gateway/th

100%|████████████████████████████████████████████| 4/4 [00:00<00:00, 109.91it/s]

[rdflib.term.URIRef('http://localhost/web-thing-gateway/things/visualizations/colored-measurement'), rdflib.term.URIRef('http://localhost/web-thing-gateway/things/visualizations/enriched-call'), rdflib.term.URIRef('http://localhost/web-thing-gateway/things/visualizations/time-series-line-chart-with-time-range-selector')]
[rdflib.term.URIRef('http://localhost/web-thing-gateway/things/visualizations/colored-measurement')]
[rdflib.term.URIRef('http://localhost/web-thing-gateway/things/visualizations/colored-measurement'), rdflib.term.URIRef('http://localhost/web-thing-gateway/things/visualizations/enriched-call'), rdflib.term.URIRef('http://localhost/web-thing-gateway/things/visualizations/time-series-line-chart-with-time-range-selector')]
[rdflib.term.URIRef('http://localhost/web-thing-gateway/things/visualizations/enriched-call')]
[rdflib.term.URIRef('http://localhost/web-thing-gateway/things/visualizations/colored-measurement'), rdflib.term.URIRef('http://localhost/web-thing-gateway/th